## Random Sampling

Select 248 random clips in total (considering 31 classes) or 8 random clips per class. Although I am including 31 classes to respect the 20% of random sampled clips I am avoinding including clips labeled as Res Junglefowl, as there are so many and I want to avoid labeling clips from this class. The same as other confounding sounds. For these I will only include them in uncertainty sampling.

Global Random Sampling:

Perform random sampling across all remaining predictions without class constraints.
Adjust sampling to exclude clips belonging to any unwanted classes (e.g., Gallus_gallus), if needed in the future.

In [1]:
import pandas as pd
from pathlib import Path
import random
from opensoundscape import Audio


/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/ml/cnn.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# These paths never change
tracker_file = '/mnt/d/retraining_BirdNET_2025/model_train/train_set/training_set_tracker.csv'
audio_files_dir = '/mnt/d/retraining_BirdNET_2025/iterative_training/input_20%/'

*Modify the paths below accordingly.*  

Remember during **iteration 1** the results will come from **model_0**. On **iteration 2** the results will come from **model_1** and so on..  

In [3]:
# Paths
annotations_file = '/mnt/d/retraining_BirdNET_2025/iterative_training/results/model_0_s1_t0.01/BirdNET_SelectionTable.txt'
output_dir = '/mnt/d/retraining_BirdNET_2025/iterative_training/segments_validation/it_1_s1_t0.01/random/'

random_clip_count = 240  # Number of random clips to sample

In [4]:
# Load BirdNET predictions
predictions = pd.read_csv(annotations_file, sep='\t', usecols=['Begin Path', 'File Offset (s)', 'Common Name', 'Confidence'])
predictions.rename(columns={
    'Begin Path': 'file_path',
    'File Offset (s)': 'offset',
    'Common Name': 'class',
    'Confidence': 'score'
}, inplace=True)

In [5]:
# Construct `offset_filename` using ONLY offset and filename with extension
predictions['offset_filename'] = predictions.apply(
    lambda row: f"{int(row['offset'])}_{Path(row['file_path']).name}", axis=1  # Keep original extension
)

In [6]:
# Extract elements 1 to 5 elements of the filename for comparison
def extract_core_filename(filename):
    parts = Path(filename).stem.split('_')  # Remove extension and split filename
    return '_'.join(parts[1:6])  # Extract elements 1 to 5 

In [7]:
# Load training set tracker
if Path(tracker_file).exists():
    tracker = pd.read_csv(tracker_file)

    # Only consider files added in iteration 1 and onwards
    tracker = tracker[tracker['iteration'] >= 1]

    # Extract core filename
    tracker['core_filename'] = tracker['file'].apply(lambda x: extract_core_filename(Path(x).name))
    tracker_filenames = tracker['core_filename'].tolist()
else:
    tracker_filenames = []

In [8]:
# Apply the same logic to predictions
predictions['core_filename'] = predictions['offset_filename'].apply(lambda x: extract_core_filename(x))

In [9]:
# Filter out unwanted classes
unwanted_classes = ["Red Junglefowl"]
# Apply both filters in one step using `~` (negation) to filter clips already in the traini set and avoid includin unwanted classes
predictions = predictions[
    (~predictions["class"].isin(unwanted_classes)) & 
    (~predictions["core_filename"].isin(tracker_filenames))
]

# Display result for verification
print(f"Filtered predictions: {len(predictions)} clips remaining")

Filtered predictions: 2516070 clips remaining


In [10]:
# Randomly sample clips
random_samples = predictions.sample(min(random_clip_count, len(predictions)), random_state=42)

# Save selected random clips
for _, row in random_samples.iterrows():
    class_name = row["class"]
    class_dir = Path(output_dir) / class_name
    class_dir.mkdir(parents=True, exist_ok=True)
    
    score = row['score']
    offset = int(row['offset'])
    filename = Path(row['file_path']).name
    output_filename = f"{score}_{offset}_{filename}"
  
            
    # Assuming clips are extracted similarly as before
    audio_path = Path(audio_files_dir) / filename
    try:
        # Extract audio segment using Opensoudscape Audio class
        audio = Audio.from_file(audio_path, offset=offset, duration=3)
        audio.save(class_dir / output_filename)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        
print("Random sampling complete. Clips saved.")


Random sampling complete. Clips saved.
